In [ ]:
import os
import warnings
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

warnings.filterwarnings("ignore")

# Load environment variables from a .env file  
load_dotenv()

In [ ]:
# Create a Language Model (LLM) client using the Azure OpenAI API  
# Set the temperature to zero to avoid randomness in the output  

llm_model = 'gpt-3.5-turbo'
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    model=llm_model,
    temperature=0
)

In [ ]:
# Define two prompt templates for different use cases  
# LLM 1: Movie Genre Recommender  
template = '''You are a movie genre recommendor. Given the user's favorite genre: {genre}, suggest three related genres.
 
Suggests genres:'''
prompt_template = PromptTemplate(input_variables=["genre"], template=template)
chain_one = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# LLM 2: Movie Recommender 
template = '''You are a movie recommender. Given the user's favorite genres: {genres}, suggest some movies that fall under these genres.
 
Suggest movies:'''
prompt_template = PromptTemplate(input_variables=["genres"], template=template)
chain_two = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# Link the two chains together in a sequential manner  
# The output of chain_one will be passed as input to chain_two  
overall_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True)

In [ ]:
# Run the overall chain with a sample input 
overall_chain.run('War')

example output:
```plaintext
> Entering new SimpleSequentialChain chain...
 1. Historical Drama - This genre often includes war as a central theme and focuses on the historical events and characters involved in conflicts.

2. Action - Action movies often feature intense battle scenes and military operations, making them a good choice for fans of war movies.

3. Thriller - Thriller movies often incorporate elements of war and espionage, creating tension and suspense in a similar way to war films.

1. Historical Drama - "Braveheart" (1995) - This film follows the story of William Wallace, a Scottish warrior who led a rebellion against the English during the First War of Scottish Independence.

2. Action - "Saving Private Ryan" (1998) - This movie is set during World War II and follows a group of soldiers as they search for a paratrooper whose brothers have been killed in action.

3. Thriller - "Zero Dark Thirty" (2012) - This film is a gripping thriller that follows the decade-long hunt for Osama bin Laden after the September 11 attacks, incorporating elements of war and espionage.

> Finished chain.
```